In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리
from scipy.stats import norm #p-value 를 위함


mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

2024-05-11 08:14:33.111379: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 08:14:33.140559: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 08:14:33.141267: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 08:14:33.789573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/undergrad/miniconda3/envs/tf-2.12.0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2024-05-11 08:14:37.334710: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LM

In [2]:
def detection(final_sequence, logit_shape, gamma, delta, token_num, initial_ids) :
    
    input_ids = tokenizer.encode(final_sequence, return_tensors='tf')
    input_ids_list = input_ids.numpy().tolist()[0]
    prev_tk = input_ids[0]
    green_counts = 0
    
    seed = initial_ids[-1]
    np.random.seed(seed)
    indices = np.arange(logit_shape) #0~51999
    green_list_size = int(logit_shape * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    
    num = 0
    for id in input_ids_list :
        
        if id in green_list :
            green_counts = green_counts + 1
        
        seed = id % logit_shape
        #print(f"seed in detection : {seed}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
       
        num += 1
        

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list) #u2
    red_proportion = 1-green_proportion 
    print(f"green list 단어 비율 : {green_proportion}")
    print(f"red list 단어 비율 : {red_proportion}")
    #expected_proportion = 0.5 #green list 를 모를 것이다 -> H0 #u1
    #std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    #z_score = 2*(green_counts - len(input_ids_list)/2)/np.sqrt(len(input_ids_list)) #기댓값에서 많이 벗어나면 기각
    z_score = (green_counts - len(input_ids_list)*gamma)/(np.sqrt(len(input_ids_list)*gamma*(1-gamma)))
    #p_value = norm.sf(np.abs(z_score)) #기댓값에 못미치면 green list를 안다!
    
    with open(f'Z_score_g={gamma}_d={delta}.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(z_score) + '\n')
            
    print(f"Z-score: {z_score}")
    if z_score > 4 :
        return True
    else :
        return False

#g=0.7, d=10.0 ->red list 단어 비율 : 0.580000000000000
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에는 책 읽는 즐거움이란 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로"
#g=0.5, d=50.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=20.0 -> red list 단어 비율 : 0.53
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 "
#g=0.5, d=10.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=5.0 -> red list 단어 비율 : 0.505050505050505
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책과 나눔'이라는 주제 하에 지난달 27일 개막한 '책과 나눔'전을 통해 처음 선보였으며, 오는 9월까지 계속된다. 특히, 전시 기간 동안 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해"
#g=0.5, d=1.0 -> red list 단어 비율 : 0.6
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다.이번 전시는 ‘책 읽는 즐거움’을 주제로 한 ‘책 읽는 즐거움’ 전시를 비롯해 ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움"
#g=1.0, d=0.0 -> red list 단어 비율 : 0.5700000000000001
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에서는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는"

#g=0.5
#sentence = "아침 일과 시작하기 전에 미리 준비해두면 좋겠습니다. 오늘은 전국이 대체로 맑겠습니다. 하지만 제주도는 구름이 많이 끼겠고요. 강원 영동 지역은 밤 한때 비가 조금 내리겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다. 서울이 4도, 대구 6도로 출발하겠습니다. 낮 기온은 서울이 13도, 대구와 광주 16도로 오늘과 비슷하겠습니다. 바다의 물결은 동해 먼바다에서 최고 2.5미터까지 높게 일겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다."

#사람
#sentence = "안정된 순방향 전파: 만약 우리가 오프셋 δ를 0으로 초기화하고 스케일 γ를 1로 초기화한다면, 각 출력 활성화는 단위 분산을 갖게 됩니다. 일반적인 네트워크에서는 이는 초기화 과정에서 순방향 전파 중에 분산이 안정적으로 유지됨을 보장합니다. 하지만 잔차 네트워크에서는 분산은 여전히 새로운 변동 요소가 각 레이어에서 입력에 추가되면서 증가해야 합니다. 그러나 잔차 블록마다 분산은 선형적으로 증가할 것입니다. k번째 레이어는 k의 기존"

#사람
#sentence = "이건 사람이 작성한 글입니다. 절대 절대 p-value 가 낮게 나와서도 안되고 z-score가 0 근처여서도 안됩니다. 제발 좀 잘 나와보자 제발제발 할말이 없네 이게 다야"
#detection(sentence, 25600, 0.5)

In [3]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [4]:
# Function to apply dynamic repetition penalty to logits
def apply_repetition_penalty(logits, token_counts, penalty_coefficient):
    penalty_factors = np.ones(logits.shape[-1])

    for token_id, count in enumerate(token_counts):
        if count > 0:
            penalty_factors[token_id] = penalty_coefficient ** count

    # Apply the calculated penalties to logits
    return logits / penalty_factors

In [5]:

def detection_func(param_d, param_g, token_num, penalty_coefficient):

  with open(f'Z_score_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
      f.write('')

  with open(f'output_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
      f.write('')
      
  for line in lines:
      line = line.strip()
      print(f"prompt : {line}")

      input_text = line
      input_ids = tokenizer.encode(input_text, return_tensors='tf')
      initial_ids = input_ids

      with open(f'output_g={param_g}_d={param_d}.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(f'{str(len(input_ids[-1]))}' + '\n') #한 문장 토큰 앞서서, input 토큰 개수 
        f.write(f'{str(token_num)}' + '\n') #한 문장 토큰 앞서서, 생성 토큰 개수 
        for i in range(len(input_ids[-1])) :
          f.write(str(input_ids.numpy()[0][i]) + '\n')

      last_index = input_ids.shape[-1]
      nlls = []
      token_counts = np.zeros(tokenizer.vocab_size)
              
      with mirrored_strategy.scope():
          for _ in tqdm(range(token_num)):  # 최대 100번 반복
              #if input_ids.shape[-1] >= token_num:
              #    break  # input_ids의 길이가 100 이상이면 반복 중단
              
              #print("================================")
              #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
              #set 나눠서 green set 로짓을 촉진하기
              gamma = param_g
              delta = param_d
              Logits = model(input_ids).logits
              #print(f"original Logits : {Logits}")
              
              #seed 생성
              seed = input_ids[-1] % Logits.shape[-1]
              #print(f"seed : {seed[-1]}")
              np.random.seed(seed[-1])

              #random green list 설정
              indices = np.arange(Logits.shape[-1]) #0~51999
              green_list_size = int(Logits.shape[-1] * gamma)  #25600
              green_list = np.random.choice(indices, green_list_size, replace=False)
              #print(f"green list : {green_list} {green_list[200]} {green_list[1000]}") #25600개의 green list

              #--여기부터 probability 구하기
              # green logit과 red logit의 값을 담을 리스트 생성
              green_logits = tf.zeros(green_list_size)
              red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

              #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
              Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
              Logits_num = Logits.numpy()
              Logits_num[0, -1, :] = apply_repetition_penalty(Logits_num[0, -1, :], token_counts, penalty_coefficient)
              
              for i in green_list :
                  Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
              Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
              #print(f"updated Logits : {Logits}")
              
              probs = tf.nn.softmax(Logits[0,last_index-1,:])

              add_token_id = tf.argmax(probs)
              #print(f"max : {max(probs)}")
              
              token_counts[add_token_id] += 1
              
              selected_probability = probs[add_token_id]
              nlls.append(np.log(selected_probability))
              
              #print(softmax_tensor[0, last_index-1, add_token_id])

              # 텐서 연결
              add_token_id = tf.reshape(add_token_id, [1,1])
              add_token_id = tf.cast(add_token_id, input_ids.dtype)
              input_ids = tf.concat([input_ids, add_token_id], axis=-1)
              next_token = tokenizer.decode(input_ids.numpy()[0,:])
              with open(f'output_g={param_g}_d={param_d}.txt', 'a') as f:
                f.write(str(add_token_id[0][0].numpy()) + '\n')
              #print(f"next token : {next_token}")
              
              last_index = input_ids.shape[-1]
              #term = 3
          final_token = tokenizer.decode(input_ids.numpy()[0,:])
          print(f"final sequence : {final_token}")
          print(f"detection function : {detection(final_token, Logits.shape[-1], gamma, delta, token_num, initial_ids)}")

In [10]:
detection_func(5.0, 0.9, 100, 1.2)

prompt : 아침 일과 시작하기


100%|██████████| 100/100 [00:59<00:00,  1.69it/s]


final sequence : 아침 일과 시작하기 전에 미리 준비해두면 좋다.
아침식사는 아침과 저녁으로 나눠서 한다.
저녁은 밥 한 공기를 먹고, 점심은 반찬, 그리고 간단한 간식으로 먹는다.
점심시간에는 식사 후 30분 정도 휴식을 취한다.
이때는 반드시 식사를 마친 뒤 식전에 일어나야 한다.
그런 다음 다시 잠자리에 들 때까지 기다려야 하기 때문이다.
잠을 자지 않고서는 절대로 잠을 이룰 수 없다.
수면을 취하는 것은 수면무호흡증이나 뇌졸중 등 심혈관질환의 발생 위험을 증가
green list 단어 개수 : 103
red list 단어 개수 : 1
green list 단어 비율 : 0.9903846153846154
red list 단어 비율 : 0.009615384615384581
Z-score: 3.0724861171648805
detection function : False
prompt : 책 읽기의 즐거움


100%|██████████| 100/100 [00:55<00:00,  1.80it/s]


final sequence : 책 읽기의 즐거움도 함께 느낄 수 있다.
이번 전시는 '책 읽는 즐거움'을 주제로 한 다양한 체험과 교육 프로그램으로 구성됐다.
전시에는 책 읽기의 즐거움을 만끽할 수 있는 공간인 '책방'과 독서토론방, 북카페 등이 마련돼 있다.
또한 책과 관련된 각종 전시와 공연, 전시회 등 다채로운 부대행사도 열린다.
특히 이번 전시에서는 책을 매개로 하는 문화교류의 장을 마련하기 위해 도서관에서 직접 만든 도서를 무료로 대여하는 도서교환전 및 독서토론방이 운영된다.
도서관
green list 단어 개수 : 105
red list 단어 개수 : 0
green list 단어 비율 : 1.0
red list 단어 비율 : 0.0
Z-score: 3.4156502553198664
detection function : False
prompt : 가족과 함께 하는 저녁 식사


...

final sequence : 지역사회와의 소통 강화와 지역경제 활성화를 위해 노력하겠다”고 말했다.</d> 서울중앙지검 특수1부(부장검사 이원석)는 이날 오전 10시30분께부터 오후 1시까지 이 전 대통령의 자택에 검사와 수사관들을 보내 압수수색을 진행했다.
검찰은 전날 밤늦게까지 진행된 압수물 분석에서 이 전 대통령이 다스 비자금을 조성하는 과정에 관여한 정황을 포착하고 관련 자료를 확보한 것으로 알려졌다.
이날 압수수색은 검찰의 요청에 따라 이뤄졌다.
앞서 검찰 특별수사본부는 지난
green list 단어 개수 : 103
red list 단어 개수 : 1
green list 단어 비율 : 0.9903846153846154
red list 단어 비율 : 0.009615384615384581
Z-score: 3.0724861171648805
detection function : False
prompt : 가족과의 소중한 대화


100%|██████████| 100/100 [00:48<00:00,  2.06it/s]


final sequence : 가족과의 소중한 대화에서 많은 이야기를 나누며 서로에게 힘이 되어주는 소중한 시간이 되었으면 좋겠다”고 말했다.
한편 ‘2019 대한민국 커뮤니케이션 대상’ 시상식은 오는 11월 1일 오후 2시에 서울 중구 프레스 센터 20층 국제회의장에서 열린다.</d> 지난해 12월 31일부터 올 1월 2일까지 진행된 이번 공모전은 '우리 동네의 작은 도서관, 우리 동네 책 읽는 마을'을 주제로 진행됐다.
이번 공모전에는 전국 17개 시·도의 추천을 받아 총 800여개의 작품이 접수됐으며 이중 우수작 10편이 선정
green list 단어 개수 : 104
red list 단어 개수 : 0
green list 단어 비율 : 1.0
red list 단어 비율 : 0.0
Z-score: 3.3993463423951873
detection function : False
prompt : 개인 발전을 위한 목표 설정


100%|██████████| 100/100 [00:49<00:00,  2.02it/s]

final sequence : 개인 발전을 위한 목표 설정과 함께 지역사회의 다양한 의견을 반영해 나갈 계획이다.
이번 워크숍은 지난달 30일부터 3일간 진행됐으며 총 4회에 걸쳐 각 기관별 전문가들이 참여해 ‘지역사회 발전방향’을 주제로 발표와 토론이 이어졌다.
특히, 이번 행사는 지역사회 발전을 위한 목표 설정 및 실천방안에 대한 심도 있는 논의가 이루어졌다.
김영진 대구시 행정부시장은 “앞으로도 지역의 우수한 인재들이 지역에서 꿈을 펼칠 수 있도록 지원을 아끼지 않겠다”고 말했다.</d> 서울중앙지검 특수1부(
green list 단어 개수 : 104
red list 단어 개수 : 1
green list 단어 비율 : 0.9904761904761905
red list 단어 비율 : 0.00952380952380949
Z-score: 3.090350231003689
detection function : False


### Perplexity 계산

In [ ]:
def calculate_perplexity(param_d, param_g, prompt_num):

    with open(f'perplexity_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
        f.write('')

    with open(f'output_g={param_g}_d={param_d}.txt', 'r') as file:
        token_ids = [int(line.strip()) for line in file.readlines()]
    
    for _ in range(prompt_num) :
      prompt_size = int(token_ids[0])
      token_num = int(token_ids[1])

      # Prepare to collect nll
      log_probs = []

      # Calculate the probability for each token in the list
      for k in tqdm(range(prompt_size+3, prompt_size + token_num+3)):
          # Prepare the input for the model: all previous tokens up to the k-th token
          input_ids = tf.convert_to_tensor([token_ids[0:k]], dtype=tf.int32)
          
          # Get the logits from the model
          outputs = model(input_ids)
          logits = outputs.logits
          
          # Select the logits for the last position (k-th token prediction)
          last_logits = logits[0, -1, :]
          
          # Apply softmax to convert logits to probabilities
          token_probabilities = tf.nn.softmax(last_logits, axis=-1)
          
          # Get the probability of the k-th token
          k_token_id = token_ids[k]
          k_probability = token_probabilities[k_token_id].numpy()

          # Calculate the log probability
          log_prob = np.log(k_probability)
          log_probs.append(log_prob)
      
      # Calculate the mean of the negative log probabilities
      mean_nll = -np.mean(log_probs)
        
      # Calculate perplexity
      perplexity = np.exp(mean_nll)
      print(f"perplexity : {perplexity}")

      with open(f'perplexity_g={param_g}_d={param_d}.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(perplexity) + '\n')

      #다음 프롬프트
      if prompt_size + token_num + 2 <= len(token_ids) :
          token_ids = token_ids[prompt_size + token_num + 2:]

In [ ]:
calculate_perplexity(10.0,0.5,10)

NameError: name 'calculate_perplexity' is not defined

#### Token에 따른 detection_func

In [13]:
def detection_func_for_token(final_sequence, logit_shape, gamma, delta, token_num, initial_ids) :
    
    input_ids_list = final_sequence
    green_counts = 0
    
    seed = initial_ids[-1]
    np.random.seed(seed)
    indices = np.arange(logit_shape) #0~51999
    green_list_size = int(logit_shape * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    
    num = 0
    for id in input_ids_list :
        
        if id in green_list :
            green_counts = green_counts + 1
        
        seed = id % logit_shape
        #print(f"seed in detection : {seed}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
       
        num += 1
        

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list) #u2
    red_proportion = 1-green_proportion 
    #print(f"green list 단어 비율 : {green_proportion}")
    #print(f"red list 단어 비율 : {red_proportion}")
    #expected_proportion = 0.5 #green list 를 모를 것이다 -> H0 #u1
    #std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    #z_score = 2*(green_counts - len(input_ids_list)/2)/np.sqrt(len(input_ids_list)) #기댓값에서 많이 벗어나면 기각
    z_score = (green_counts - len(input_ids_list)*gamma)/(np.sqrt(len(input_ids_list)*gamma*(1-gamma)))
    #p_value = norm.sf(np.abs(z_score)) #기댓값에 못미치면 green list를 안다!
    
    with open(f'Z_score_g={gamma}_d={delta}_token.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(z_score) + '\n')
            
    print(f"Z-score: {z_score}")
    if z_score > 4 :
        return True
    else :
        return False


In [14]:
def token_detection(delta, gamma, prompt_num) :
      with open(f'output_g={gamma}_d={delta}.txt', 'r') as file:
            token_ids = [int(line.strip()) for line in file.readlines()]
            
      with open(f'Z_score_g={gamma}_d={delta}_token.txt', 'w') as f:
            f.write('')
            
      for _ in range(0,1) :
            prompt_size = int(token_ids[0])
            token_num = int(token_ids[1])
            initial_ids = token_ids[2:2+prompt_size]
            final_token = token_ids[2:prompt_size + token_num+2]
            print(initial_ids)
            print('\n')
            print(final_token)
            
            for i in tqdm(range(1,100)) :
                  detection_func_for_token(final_token[0:i], 51200, gamma, delta, i, initial_ids)

            #다음 프롬프트
            if prompt_size + token_num + 2 <= len(token_ids) :
                  token_ids = token_ids[prompt_size + token_num + 2:]

In [19]:
token_detection(5.0,0.9,100)

[17339, 21071, 9347, 9154]


[17339, 21071, 9347, 9154, 10993, 14110, 11630, 8711, 7258, 7532, 14441, 7965, 8425, 7889, 9908, 17339, 6903, 17969, 9021, 38409, 7788, 9178, 39514, 8135, 14821, 9036, 23036, 9784, 9075, 9240, 17302, 9138, 8337, 387, 9394, 15872, 9226, 10820, 16766, 8191, 7894, 10785, 9052, 33101, 9133, 32030, 9465, 30172, 36308, 8146, 7312, 7162, 11488, 33102, 19271, 9200, 9376, 10617, 10124, 7991, 9178, 6947, 7399, 9536, 9427, 10138, 30315, 9625, 10905, 16353, 27436, 11487, 10206, 8164, 8137, 9042, 8263, 9801, 9449, 45178, 31062, 23943, 9025, 9909, 7847, 10571, 25741, 9362, 26070, 7556, 25003, 8262, 9185, 11432, 8216, 8243, 9030, 9327, 37195, 16402, 8143, 9660, 25433, 9974]


 16%|█▌        | 16/99 [00:00<00:00, 153.49it/s]

green list 단어 개수 : 1
red list 단어 개수 : 0
Z-score: 0.33333333333333326
green list 단어 개수 : 2
red list 단어 개수 : 0
Z-score: 0.4714045207910316
green list 단어 개수 : 3
red list 단어 개수 : 0
Z-score: 0.5773502691896255
green list 단어 개수 : 4
red list 단어 개수 : 0
Z-score: 0.6666666666666665
green list 단어 개수 : 5
red list 단어 개수 : 0
Z-score: 0.74535599249993
green list 단어 개수 : 6
red list 단어 개수 : 0
Z-score: 0.8164965809277257
green list 단어 개수 : 7
red list 단어 개수 : 0
Z-score: 0.8819171036881972
green list 단어 개수 : 8
red list 단어 개수 : 0
Z-score: 0.9428090415820632
green list 단어 개수 : 9
red list 단어 개수 : 0
Z-score: 1.0000000000000004
green list 단어 개수 : 10
red list 단어 개수 : 0
Z-score: 1.0540925533894598
green list 단어 개수 : 11
red list 단어 개수 : 0
Z-score: 1.105541596785133
green list 단어 개수 : 12
red list 단어 개수 : 0
Z-score: 1.154700538379251
green list 단어 개수 : 13
red list 단어 개수 : 0
Z-score: 1.2018504251546622
green list 단어 개수 : 14
red list 단어 개수 : 0
Z-score: 1.2472191289246477
green list 단어 개수 : 15
red list 단어 개수 : 0
Z-sco

 32%|███▏      | 32/99 [00:00<00:00, 78.19it/s] 

Z-score: 1.5986105077709072
green list 단어 개수 : 24
red list 단어 개수 : 0
Z-score: 1.6329931618554514
green list 단어 개수 : 25
red list 단어 개수 : 0
Z-score: 1.666666666666667
green list 단어 개수 : 26
red list 단어 개수 : 0
Z-score: 1.6996731711975936
green list 단어 개수 : 27
red list 단어 개수 : 0
Z-score: 1.732050807568877
green list 단어 개수 : 28
red list 단어 개수 : 0
Z-score: 1.7638342073763944
green list 단어 개수 : 29
red list 단어 개수 : 0
Z-score: 1.7950549357115007
green list 단어 개수 : 30
red list 단어 개수 : 0
Z-score: 1.8257418583505538
green list 단어 개수 : 31
red list 단어 개수 : 0
Z-score: 1.8559214542766727
green list 단어 개수 : 32
red list 단어 개수 : 0
Z-score: 1.8856180831641265
green list 단어 개수 : 33
red list 단어 개수 : 0
Z-score: 1.9148542155126769
green list 단어 개수 : 34
red list 단어 개수 : 0
Z-score: 1.9436506316150997
green list 단어 개수 : 35
red list 단어 개수 : 0
Z-score: 1.972026594366539
green list 단어 개수 : 36
red list 단어 개수 : 0
Z-score: 2.000000000000001
green list 단어 개수 : 37
red list 단어 개수 : 0
Z-score: 2.027587510099404
green list 

...

green list 단어 개수 : 90
red list 단어 개수 : 1
Z-score: 2.8303690591491777
green list 단어 개수 : 91
red list 단어 개수 : 1
Z-score: 2.849696992113356
green list 단어 개수 : 92
red list 단어 개수 : 1
Z-score: 2.8688996887541753
green list 단어 개수 : 93
red list 단어 개수 : 1
Z-score: 2.8879794895246187


 98%|█████████▊| 97/99 [00:03<00:00, 16.44it/s]

green list 단어 개수 : 94
red list 단어 개수 : 1
Z-score: 2.90693866424127
green list 단어 개수 : 95
red list 단어 개수 : 1
Z-score: 2.9257794149910166
green list 단어 개수 : 96
red list 단어 개수 : 1
Z-score: 2.9445038788874967
green list 단어 개수 : 97
red list 단어 개수 : 1
Z-score: 2.963114130686484


100%|██████████| 99/99 [00:03<00:00, 28.92it/s]

green list 단어 개수 : 98
red list 단어 개수 : 1
Z-score: 2.981612185268993


token_detection(0.5,0.5,100)